In [ ]:
# Using python libraries to automate process of selecting dates and scrap the daily data from feb 2020 to Apr 2020.

# Import Required Libraries
import time
import pandas as pd
from bs4 import BeautifulSoup
# Import webdriver to initialise a browser
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import openpyxl

# Read dates from Excel File into a list (if you are running on your computer, import the Excel sheet with name dates.101 that given in the main folder)
# dates.101.xlsx file contain the data of dates that we want to scrape from website. Here 1st feb 2020 to 30th april 2020.
df = pd.read_excel(r"dates.101.xlsx")
dates = df['Unnamed: 8']
dates = list(dates)

# URL to Scrape
url = "https://fcainfoweb.nic.in/reports/report_menu_web.aspx"

# Initialize webdriver and put the path where download the driver
driver = webdriver.Chrome()
driver.maximize_window()

# Launch Chrome and pass the url
driver.get(url)

# Loop through the dates and scrape the data for each date
for date in dates:
    # Select the "Price Report" Option
    driver.find_element(by=By.XPATH, value="//label[normalize-space()='Price report']").click()

    # Select the "Daily Prices" option from the dropdown
    truck = driver.find_element(by=By.ID,value="ctl00_MainContent_Ddl_Rpt_Option0")
    truck1 = Select(truck)
    truck1.select_by_visible_text("Daily Prices")

    # Enter the date and submit the form
    dr=driver.find_element(by=By.XPATH,value="//input[@id='ctl00_MainContent_Txt_FrmDate']").send_keys(date)
    driver.find_element(by=By.XPATH,value="//input[@id='ctl00_MainContent_btn_getdata1']").click()

    # Wait for the page to load
    time.sleep(1)

    # Extract the data from the table and convert to a Pandas DataFrame
    webtable_df= pd.read_html(driver.find_element(by=By.XPATH,value="//table[@id='gv0']").get_attribute('outerHTML'))[0]

    # Save the DataFrame as a CSV and Excel file
    webtable_df.to_csv(f'{date[0:2]}.{date[3:5]}.2020.csv')
    read_file = pd.read_csv (f'{date[0:2]}.{date[3:5]}.2020.csv')
    read_file.to_excel (f'{date[0:2]}.{date[3:5]}.2020.xlsx', index=None, header=True)
    # Remove the temporary CSV file
    os.remove(f'{date[0:2]}.{date[3:5]}.2020.csv')
    # Go back to the form
    driver.find_element(by=By.XPATH,value="//input[@id='btn_back']").click()

# Combine the sheets into a single workbook
# Select the local path in your computer to save the excel sheet (Path shoudl be the same where your all single files stored)
path = r"C:\Users\SAHIL\PycharmProjects\pythonProject\ISB_Work"
target_wb = openpyxl.Workbook()

for filename in os.listdir(path):
    if filename.endswith(".xlsx"):
        source_wb = openpyxl.load_workbook(os.path.join(path, filename), data_only=True)
        for sheet in source_wb:
            target_sheet = target_wb.create_sheet(f"{filename}")
            for row in sheet.iter_rows(values_only=True):
                target_sheet.append(row)
        source_wb.close()

# Save the Excel Workbook
target_wb.save("combined_sheets.xlsx")
